In [1]:
# Импорт библиотек
import os
import time
import GTU
import KU_TU
import mat_properties as prop
import numpy as n
import pandas as pd
import SP
from calculate_CCGT_PKM import Calculate_CCGT_PKM_iter
from scipy.optimize import root

# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)
GTU_ISO = pd.read_excel("input.xlsx", sheet_name="ISO", index_col=0)
GTU_input = pd.read_excel("input.xlsx", sheet_name="GTU_input", index_col=0)
# рабочая таблица (=номинал в 1 итерации)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
# рабочая таблица показателей блоков
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)
electric = pd.read_excel("blocks.xlsx", sheet_name="electric", index_col=0)
accumulation = pd.read_excel("blocks.xlsx", sheet_name="accumulation", index_col=0)
############################################################
# Теплосеть и перекидка температуры воздуха
gas_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
water_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
Tnv = gas_streams.at["AIR", "T"]
water_streams.at["SWIN", "T"] = SP.Tset(Tnv)[1]
water_streams.at["SWOUT", "T"] = SP.Tset(Tnv)[0]
water_streams.at["SWIN-TURB", "T"] = water_streams.at["SWIN", "T"]
water_streams.at["SWIN-TURB", "G"] = water_streams.at["SWIN", "G"]
water_streams.at["SP2-WOUT", "T"] = water_streams.at["SWOUT", "T"]
############################################################

# Основные эффективности оборудования
KPD_PN = 0.8074
KPD_KN = 0.75
KPD_to = 0.99
KPD_SP = 0.99


# Параметры, отвечающие за процесс расчета
Calcmethod = "hybr"
Calctolerance = 10**-2
Maxiterations_KU_TU = 20
Maxiterations_cotel = 10
Maxiterations_turbine = 20

# Параметры режима работы ПГУ
# Расчет для минимума нагрузки
Сalculate_minimum = True


# Расчет для работы с теплофикацией
# Teplo = int(False)
# Отбор пара высокго давления или доля или кг/с
steamVD_fraction_to_turbine = 1


# Состав газов в номинале в ГТУ
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
fractiongas0 = list(gas_streams0.loc["GTU-PEVD", "N2":"Ar"])
gas0 = prop.Materials_prop(
    gasmix,
    fractiongas0,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
# Задание энтальпий газа в номинальном режиме
Temperatures = gas_streams0.loc["GTU-KU":"GPK-out", "T"]
Pressure = gas_streams0.loc["GTU-KU", "P"]
gas_streams0.loc["GTU-KU":"GPK-out", "H"] = list(
    map(lambda x: gas0.p_t(Pressure, x)["h"], Temperatures)
)


#####################Максимов#####################
water_streams.at["SWIN", "H"] = water.p_t(1, water_streams.at["SWIN", "T"])["h"]
water_streams.at["SWOUT", "H"] = water.p_t(1, water_streams.at["SWOUT", "T"])["h"]

# Время зарядки/разрядки, часы
time_ac = 4
# Время ожидания, часы
time_jdat = 12
# Конструкция аккумулятора
constr = {
    "Diametr": 20,
    "kolichestvo": 8,
    "Visota": 20,
    "lambda_min_vata": 0.045,
    "delta_min_vata": 0.01,
}

syngas_streams = pd.read_excel("streams.xlsx", sheet_name="syngas", index_col=0)
##################################################


############################################################
# Задание ГТУ
Gas_turbine = GTU.gtu(GTU_ISO, "GTU-KU")

Iter_pkm = 20
pkm_pgu_tol = 10**-2




In [2]:
from calculate_CCGT_PKM import Calculate_CCGT_PKM_iter

PKM_zaryad = False
PKM_razryad = True

arguments_all_it = [
    Maxiterations_KU_TU,
    Maxiterations_cotel,
    Maxiterations_turbine,
    gas_streams0,
    water_streams0,
    GTU_ISO,
    GTU_input,
    gas_streams,
    water_streams,
    heaters,
    electric,
    Gas_turbine,
    gas0,
    water,
    PKM_zaryad,
    PKM_razryad,
    syngas_streams,
    Calcmethod,
    Calctolerance,
    KPD_PN,
    KPD_KN,
    KPD_to,
    KPD_SP,
    steamVD_fraction_to_turbine,
    accumulation,
    time_ac,
    constr,
    time_jdat,
]
New_Iter_pkm=20
pkm_pgu_tol=10**-2
CCGT=Calculate_CCGT_PKM_iter(arguments_all_it, New_Iter_pkm, pkm_pgu_tol)

Тепла в ГВТО хватает на теплофикацию
Не сходится баланс камеры сгорания 41.958396943850346 5.300648808770347e-12 41.958396943845045
Рассчитана пиковая ГТУ, расход воздуха: 932.7595236233258
Время 1 итерации расчета КУ+ТУ:---  10.3 сек. ---
Достигнуто максимальное количество итераций котла-утилизатора:3
Время 2 итерации расчета КУ+ТУ:---  28.3 сек. ---
Время 3 итерации расчета КУ+ТУ:---  35.3 сек. ---
Достигнуто максимальное количество итераций котла-утилизатора:3
Время 4 итерации расчета КУ+ТУ:---  56.4 сек. ---
Время 5 итерации расчета КУ+ТУ:---  64.1 сек. ---
Время 6 итерации расчета КУ+ТУ:---  69.2 сек. ---
Время 7 итерации расчета КУ+ТУ:---  74.9 сек. ---
Время 8 итерации расчета КУ+ТУ:---  82.4 сек. ---
Время 9 итерации расчета КУ+ТУ:---  85.7 сек. ---
Время 10 итерации расчета КУ+ТУ:---  87.1 сек. ---
Расчет КУ+ПТУ окончен.
Максимальная погрешность определения расходов при расчете КУ+ПТУ 1.808051185238631e-14
Pnd_it [0.70028, 0.70482, 0.70155, 0.69866, 0.69786, 0.69771, 0.69772, 

In [3]:
water_streams

,T,P,H,G,S,X
AIR,-29.000000,0.100000,NaN,NaN,NaN,NaN
PEVD-DROSVD,499.800329,8.290802,3395.500509,60.874797,NaN,NaN
IVD-PEVD,300.174441,8.609030,2749.302552,60.874797,NaN,NaN
EVD-IVD,299.294556,8.609030,1340.934828,60.874797,NaN,NaN
PEN-EVD,168.556338,8.609030,717.145213,60.874797,NaN,NaN
BND-PEN,165.144861,0.703455,697.868168,60.874797,NaN,NaN
PPND-DROSND,210.178733,0.697793,2868.018123,15.197635,NaN,NaN
IND-PPND,165.144861,0.703455,2762.960612,15.197635,NaN,NaN
GPK-IND,162.221867,0.703455,685.162723,76.072432,NaN,NaN
GPK-REC,162.221867,0.703455,685.162723,89.958043,NaN,NaN


In [4]:
syngas_streams

,T,P,H,G,N2,O2,CO2,Ar,H2O,CH4,H2,CO
REF-COOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
COOL-HTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HTS-HTSCOOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HTSCOOL-Separ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Separ-SGaccum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SGaccum-Separacc,-21.803926,2.0,998.444452,365775.995050,0.000000,0.000000,0.168738,0.000000,0.050357,0.097508,0.681387,0.002009
Separacc-Sepout,-21.803926,2.0,879.636427,336452.977654,0.000000,0.000000,0.177686,0.000000,0.000000,0.102679,0.717520,0.002115
Sepout-COMB,-21.803926,2.0,879.636427,22.283679,0.000000,0.000000,0.177686,0.000000,0.000000,0.102679,0.717520,0.002115
Sepout-GTU,-21.803926,2.0,879.636427,1.081111,0.000000,0.000000,0.177686,0.000000,0.000000,0.102679,0.717520,0.002115
AIR-COMP,15.000000,0.1,293.187169,932.759524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
from PKM import PKM_all

Gst = 1
pkm = PKM_all.calc(True,gas_streams,syngas_streams,water_streams,water_streams0,heaters, Gst)

TypeError: PKM_all.calc() takes 6 positional arguments but 7 were given